# Проект по SQL

## Цель исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Задача:

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
#!pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Таблицы:

### Таблица books

In [3]:
query = '''
SELECT *
FROM books
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица publishers

In [4]:
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица authors

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица ratings

In [6]:
query = '''
SELECT *
FROM ratings
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews

In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


На первый взгляд данные в таблицах корректны, заголовки столбцов удобны, а все значения соответствуют содержанию столбцов.

## Решение задания

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query='''
SELECT COUNT(book_id) AS "books after 01.01.2000"
FROM books
WHERE publication_date>'2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,books after 01.01.2000
0,819


После 1 января 2000 года вышло 819 книг. То есть выпускают примерно по 41 книге в год, наверно, это небольшое количество.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query='''
SELECT ratings.book_id AS book_id,
    COUNT(reviews.book_id) AS count_reviews, 
    AVG(ratings.rating) AS average_rating
FROM
    ratings
    LEFT JOIN reviews ON (reviews.book_id = ratings.book_id
            AND reviews.username = ratings.username)
GROUP BY 
    ratings.book_id
ORDER BY
    ratings.book_id;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,count_reviews,average_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


Данный запрос выводит таблицу, в которой каждому book_id соответствует количество обзоров и средний рейтинг.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query='''
SELECT publishers.publisher, 
    COUNT(books.book_id) AS count_books
FROM
    publishers
    INNER JOIN books ON books.publisher_id=publishers.publisher_id 
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY COUNT(books.book_id) DESC
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penquin Books. Прогуглив, мы узнали, что это достаточно старое и серьезное издательство, поэтому неудивительно, что оно на первом месте.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query='''
SELECT
    authors.author,
    AVG(SUB.average_rating) AS avg
FROM
    (SELECT
        book_id AS book_id,
        COUNT(rating_id) AS rating_counts,
        AVG(rating) AS average_rating
    FROM
        ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50) 
    AS SUB
INNER JOIN books ON books.book_id = SUB.book_id
INNER JOIN authors ON authors.author_id = books.author_id
GROUP BY authors.author
ORDER BY avg DESC
;
'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самый популярный автор - Джоан Роулинг, что и неудивительно из-за цикла книг про Гарри Поттера.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query='''
SELECT AVG(SUB.count)
FROM
    (SELECT COUNT(*)
    FROM reviews
    WHERE username IN
        (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating) > 50)
    GROUP BY username) AS SUB;
'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Пользователи, которые поставили больше 50 оценок, в среднем пишут 24 обзора. Это говорит о том, что чем активнее пользователь применяет приложение, тем больше желает высказать свое мнение.